In [1]:
import pandas as pd
import re
import numpy as np
import requests
import glob
import os
from tqdm import tqdm
from janitor import clean_names
from dicts import dict_columns_rrhh_download, dict_columns_select_download, path
from openpyxl import load_workbook
from datetime import datetime

In [ ]:
def download_excel(files, path_download):
    """
        Extrae código, razon_social, rut y declaración de retención de impuestos y pagos provisionales codigos: 538, 142 y 020

        Args:
            files (str): path al archivo a extraer
    """
    
    try: 

        sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        df = pd.read_excel(files,
                    sheet_name=nombre_hoja).clean_names()
        try:
            try:

                patron = "company_name|application_name"  # Patrón para identificar la fila

                fila_nombres = df[df.apply(lambda row: row.str.contains(patron, case=False, na=False)).any(axis=1)].index[0]

            except:
                patron = "application_name"  # Patrón para identificar la fila

                fila_nombres = df[df.eq(patron).any(axis=1)].index[0]
            
            df = pd.read_excel(files,
                    sheet_name=nombre_hoja,
                    header=fila_nombres+1).clean_names()

            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
                
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)

        except:
            inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
            df = df.rename(columns=inverted_dict_column)

            df = df.filter(items=dict_columns_select_download)
    
    except:

        # sheet_patron = r"BD_Postulación|BD_postulacion|BD postulacion|BD postulación"

        # nombre_hoja = next(sheet for sheet in pd.ExcelFile(files).sheet_names if re.search(sheet_patron, sheet, flags=re.IGNORECASE))

        # df = pd.read_excel(files, sheet_name=sheet_patron).clean_names()

        df = pd.read_excel(files).clean_names()

        inverted_dict_column = {value: key for key, values in dict_columns_rrhh_download.items() for value in values}
            
        df = df.rename(columns=inverted_dict_column)

        #df = df[(df.application_sent == 'si') & (~df.plan_trabajo_presupuesto_rrhh.isna())]

        try:
            df = df.filter(items=dict_columns_select_download)

        except:
            df = df.loc[:, dict_columns_select_download]

    for index, row in df.iterrows():
        codigo = str(row['codigo'])
        carpeta_destino = f'{path_download}/{codigo}'

        if not os.path.exists(carpeta_destino):
            os.makedirs(carpeta_destino)

        hipervinculo = row['plan_trabajo_presupuesto_rrhh']
    
        nombre_archivo = codigo + '.xlsx'
        
        ruta_archivo = os.path.join(carpeta_destino, nombre_archivo)
        
        if os.path.exists(ruta_archivo):
            continue

        try:
            response = requests.get(hipervinculo)
            with open(ruta_archivo, 'wb') as file:
                file.write(response.content)
        except Exception as e:
            print(f"Error al descargar el archivo desde {hipervinculo}: {str(e)}")
            continue

In [ ]:
path = r'data\2.2023\CH2'

In [ ]:
def rrhh_scrap(path, año=2023):
    """
        Recorre carpetas por año para extraer RRHH reportados por beneficiarios en base al código del proyecto

        Args:
            path (str): path a los archivos que contienen las BD de postulación con los archivos de RRHH postulados.
            año (int): año de la convocatoria
    """
    
    for concurso in tqdm(glob.iglob(f'{path}/*')):
        print(concurso)
        download_excel(concurso, path_download = 'output/2.2023/IATS/Plan de trabajo')

In [ ]:
rrhh_scrap(path)

# Cambios nombres de hoja de presupuesto

In [ ]:
path = r'data/2.2023/CH2/2.2023 Capital Humano Gestión de Evaluación.xlsx'

In [ ]:
df = pd.read_excel(path).clean_names()

In [ ]:
df.columns.to_list()

## **Proceso de Inserción de datos asociados a PPTO y gasto I+D efectivo**

A continuación, se implementará un sistema que nos permite obtener información relevante del presupuesto de los proyectos y  del aporte que éste realiza en el item I+D.

Entre los hitos fundamentales que se realizarán son:
- El código del proyecto obtenido de Charly y SGP.
- Aporte Innova Chile (Subsidio) $
- Aporte Beneficiaria (Pecuniario) $
- Aporte Beneficiaria (Valorado) $ 
- Aporte Asociados (Pecuniario) $ 
- Aporte Asociados (Valorado) $ 
- Total ($). Corresponde al monto total del proyecto
- Gasto de Inversión
- Gastos de Administración
- Gasto I+D RECHAZADO. Corresponde a la suma total del gasto I+D rechazado
- Gasto I+D efectivo. El valor final del proyecto I+D
- Porcentaje del valor del proyecto I+D declarado en relación al Total($)

### **Pasos de ejecución**

#### **1-. Mostrar la ruta absoluta según perfil**
 - Ejecutar celda para mostrar la ruta principal según perfil de usuario.
 - El resultado mostrará la ruta  del usuario registrado.

In [124]:
# Se obtiene la ruta principal, que es llamada del directorio. Acá se pueden crear n rutas para distintos perfiles.
path['esteban']

'C:/Users/esteban.berrios/OneDrive - corfo.cl/'

#### **2-. Guardar la ruta en donde se encuentran guardados los proyectos**
- Ejecutar celda para llamar la ruta absoluta asociada a mi perfil, y concatenarla con la ruta relativa asociada al directorio donde están los proyectos.
- En la variable input_path quedará guardada la ruta de acceso a los proyectos.

In [125]:
# Tabla con presupuestos. El códigop permite generar nueva ruta llamada input_path que apunta a un directorio específico
# dentro de la estructura de directorios en path['esteban']. Esto facilita la manipulación y navegación en el sistema de archivos dentro de ese directorio en particular.
ruta = os.path.abspath(path['esteban'])
input_path = os.path.join(ruta, "extract_pdt/output/2.2023/IATS/Plan de trabajo")


#### **3-. Mostrar la nueva ruta concatenada correspondiente al directorio en donde se guardan los proyectos**
- La ejecución de la celda mostrará la ruta asociada al directorio en donde se guardan los proyectos en formato .xlsx.

In [126]:
input_path #Muestra la nueva ruta.

'C:\\Users\\esteban.berrios\\OneDrive - corfo.cl\\extract_pdt/output/2.2023/IATS/Plan de trabajo'

#### **4-. Mostrar el acceso a la carpeta asociada a los proyectos**
- El resultado de la ejecución de esta celda, permite mostrar el llamado del primer proyecto que se encuentra en la carpeta.

In [127]:
#  Se define la variable folder como un vacío, y  con la siguiente línea de toma el primer archivo encontrado en una carpeta específica (dada por input_path y folder), 
# elimina la extensión ".xlsx" del nombre de ese archivo y devuelve el resultado, que probablemente sea utilizado como un identificador o código para ese archivo en particular.
folder = ""
os.listdir(os.path.join(input_path,folder))[0].replace(".xlsx","")

'23IATS-248256'

#### **5-.Guardar en un Primer Data Frame la información correspondiente a los indicadores de presupuesto de los proyectos**
A partir de la ejcución de esta celda que contiene la estructura de código podrá obtener un Data Frame que tendrá de todos los proyectos:
- El código obtenido de Charly y SGP.
- Aporte Innova Chile (Subsidio) $
- Aporte Beneficiaria (Pecuniario) $
- Aporte Beneficiaria (Valorado) $ 
- Aporte Asociados (Pecuniario) $ 
- Aporte Asociados (Valorado) $ 
- Total ($). Corresponde al monto total del proyecto
- Gasto de Inversión
- Gastos de Administración

El resultado se guardará en el data frame llamado df_presupuesto.



In [128]:
%%capture
%%time
df_presupuesto = pd.DataFrame()

for folder in os.listdir(input_path):
    folder_path = os.path.join(input_path, folder)
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]

        for excel_file in excel_files:
            file = os.path.join(folder_path, excel_file)
            codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

            # Lectura del archivo Excel
            worksheet = pd.read_excel(file, sheet_name="RESUMEN PPTO")

            # Filtrar y procesar los datos iniciales del presupuesto sin la verificación del aporte I+D verificado.
            worksheet = worksheet.iloc[7:13, 1:]
            worksheet.columns = worksheet.iloc[0]
            worksheet = worksheet[1:].reset_index(drop=True)
            # String que captura nombre de variable Cuenta (Financiable o Presupuestable)
            str_cuenta_financiable = worksheet.columns[worksheet.columns.str.contains("Cuenta")].values[0]
            
            worksheet_row = worksheet[worksheet[str_cuenta_financiable] == "TOTAL ($)"]
            fila_gasto_inversion = worksheet[worksheet[str_cuenta_financiable] == "Gasto de Inversión"]
            indice_gasto_inversion = fila_gasto_inversion.index[0]  # Obtenemos el índice de la fila "Gasto de Inversión"
            registro_gasto_inversion = fila_gasto_inversion.iloc[0,1]
            fila_gasto_administracion = worksheet[worksheet[str_cuenta_financiable] == "Gastos de Administración"]
            indice_gasto_administracion = fila_gasto_administracion.index[0]  # Obtenemos el índice de la fila "Gasto de Inversión"
            registro_gasto_administracion = fila_gasto_administracion.iloc[0,1]


            # Anexar los datos al DataFrame df_final
            worksheet_row.insert(0, "codigo", codigo) # Agregar la columna 'codigo' en la primera ubicación a la izquierda
            worksheet_row.insert(worksheet_row.columns.__len__(),"Gasto de Inversión",registro_gasto_inversion)
            worksheet_row.insert(worksheet_row.columns.__len__(),"Gastos de Administración",registro_gasto_administracion)
            worksheet_row.drop(columns=[str_cuenta_financiable], inplace=True) # Eliminar la columna 'Cuentas Financiables'
            df_presupuesto = pd.concat([df_presupuesto, worksheet_row])

# Reiniciar el índice del DataFrame resultante
df_presupuesto.reset_index(drop=True, inplace=True)

# Mostrar las primeras filas del DataFrame
df_presupuesto.head()

#### **6-.Guardar en un Segundo Data Frame la información correspondiente al valor I+D rechazado por proyecto**
A partir de la ejecución de esta celda que contiene la estructura de código podrá obtener un Segundo Data Frame que tendrá de todos los proyectos:
- El gasto I+D RECHAZADO. Corresponde a la suma total del gasto I+D rechazado

El resultado se guardará en el data frame llamado df_imasd.

##### **IMPORTANTE: ESTA CELDA SÓLO SE OCUPARÁ CUANDO LOS INSTRUMENTOS CONSIDEREN EL APORTE I+D!**

In [129]:
%%capture
%%time
# Sólo concursos con I+D
df_imasd = pd.DataFrame()
for folder in os.listdir(input_path):
    folder_path = os.path.join(input_path, folder)
    if os.path.isdir(folder_path):
        excel_files = [file for file in os.listdir(folder_path) if file.endswith(".xlsx")]

        for excel_file in excel_files:
            file = os.path.join(folder_path, excel_file)
            codigo = os.listdir(os.path.join(input_path, folder))[0].replace(".xlsx", "")

            # Lectura del archivo Excel
            worksheet = pd.read_excel(file, sheet_name="PLAN DE TRABAJO")

            # Filtrar y procesar los datos iniciales del presupuesto sin la verificación del aporte I+D verificado.
            valorimasd = 0
            # Selecciona las filas y columnas necesarias
            worksheet = worksheet.iloc[11:,5:]
            worksheet['codigo']=codigo
            worksheet_filtered = worksheet[worksheet['Unnamed: 5'].str.upper() == "NO"]

            # Sumar todos los valores en las columnas "Unnamed: 12" a "Unnamed: 16"
            total_sum = worksheet_filtered[["Unnamed: 12", "Unnamed: 13", "Unnamed: 14", "Unnamed: 15", "Unnamed: 16"]].sum().sum()

            # Crear un nuevo DataFrame con el resultado
            resultado_df = pd.DataFrame({'codigo':[codigo],'Gasto I+D RECHAZADO': [total_sum]})

            # Agregar el resultado al DataFrame principal (df_imasd)
            df_imasd = df_imasd.append(resultado_df, ignore_index=True)
            df_imasd['Gasto I+D RECHAZADO'] = df_imasd['Gasto I+D RECHAZADO'].astype('int64')
df_imasd.head()

#### **7-.Concatenar Data Frames asociados a valores de presupuestos y el gasto I+D rechazado.**
A partir de la ejecución de esta celda que contiene la estructura de código podrá obtener un data frame que tendrá:
- Los datos generales del presupuesto y el gasto I+D rechazado a cada proyecto.

El resultado se guardará en el data frame llamado df_resultado.

##### **IMPORTANTE: ESTA CELDA SÓLO SE OCUPARÁ CUANDO LOS INSTRUMENTOS CONSIDEREN EL APORTE I+D!**

In [130]:
df_resultado = pd.concat([df_presupuesto.set_index('codigo'), df_imasd.set_index('codigo')], axis=1)
df_resultado.head()

,Aporte Innova Chile\n(Subsidio) $,Aporte Beneficiaria\n(Pecuniario) $,Aporte Beneficiaria\n(Valorado) $,Aporte Asociados\n(Pecuniario) $,Aporte Asociados\n(Valorado) $,Total ($),Gasto de Inversión,Gastos de Administración,Gasto I+D RECHAZADO
codigo,,,,,,,,,
23IATS-248256,325901727.8,129335026.2,766443,2350000,9599904,467953101,53991000,20832000,138454567
23IATS-248257,239218092.371134,65000000,37690148.4,0,0,341908240.771134,0,0,115298728
23IATS-248258,178765000,41247360,40950819,0,0,260963179,0,0,110863179
23IATS-248259,349440000,278680000,249312000,0,0,877432000,0,0,2000000
23IATS-248260,598992000,125918000,25790000,26280000,80280000,857260000,102500000,1260000,28106885


#### **8-.Crear una nueva columna en el Data Frame que contendrá información del gasto I+D efectivo**
A partir de la ejcución de esta celda se podrá obtener:
- Un data frame actualizado que contendrá en la última columna el ítem Gasto I+D efectivo.
- El resultado proviene de la diferencia entre el valor total del proyecto menos el gasto I+D rechazado.

El resultado quedará actualizado en la variable df_resultado.

##### **IMPORTANTE: ESTA CELDA SÓLO SE OCUPARÁ CUANDO LOS INSTRUMENTOS CONSIDEREN EL APORTE I+D!**

In [131]:
df_resultado['Gasto I+D efectivo'] = df_resultado['Total ($)']- df_resultado['Gasto I+D RECHAZADO']
df_resultado.head()

,Aporte Innova Chile\n(Subsidio) $,Aporte Beneficiaria\n(Pecuniario) $,Aporte Beneficiaria\n(Valorado) $,Aporte Asociados\n(Pecuniario) $,Aporte Asociados\n(Valorado) $,Total ($),Gasto de Inversión,Gastos de Administración,Gasto I+D RECHAZADO,Gasto I+D efectivo
codigo,,,,,,,,,,
23IATS-248256,325901727.8,129335026.2,766443,2350000,9599904,467953101,53991000,20832000,138454567,329498534
23IATS-248257,239218092.371134,65000000,37690148.4,0,0,341908240.771134,0,0,115298728,226609512.771134
23IATS-248258,178765000,41247360,40950819,0,0,260963179,0,0,110863179,150100000
23IATS-248259,349440000,278680000,249312000,0,0,877432000,0,0,2000000,875432000
23IATS-248260,598992000,125918000,25790000,26280000,80280000,857260000,102500000,1260000,28106885,829153115


#### **9-.Crear una nueva hoja de trabajo o worksheet en el documento xlsx que contendrá el resultado final del data frame importado a excel**
A partir de la ejcución de esta celda se podrá obtener:
-La importación del resultado final del Data Frame a un worksheet definido según la veriable del archivo excel y el nombre de la hoja.

In [14]:
nombre_archivo_excel = 'data/2.2023/IATS/2.2023 IATS Gestión de Evaluación.xlsx'
nombre_hoja_excel = 'PRESUPUESTO'  # Cambia 'Hoja1' al nombre de tu hoja específica

# Crea un objeto ExcelWriter para escribir en el archivo Excel
with pd.ExcelWriter(nombre_archivo_excel, engine='openpyxl', mode='a') as writer:
    # Carga el DataFrame en la hoja de Excel especificada
    df_resultado.reset_index().to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Usar carga de información para instrumentos que consideran aporte I+D
    #df_presupuesto.reset_index(drop=True).to_excel(writer, sheet_name=nombre_hoja_excel, index=False)# Al resetear el indice, automáticamente queda asociado como columna. SE DEBE USAR PARA INSTRUMENTOSQUE NO CONTEMPLEN APORTE I+D.


